In [2]:
# 1. Install (or ensure) everything in the current kernel
%pip install --upgrade imageio imageio-ffmpeg ultralytics

Note: you may need to restart the kernel to use updated packages.


In [27]:
# 2. Imports and extract frames via imageio
import imageio
from pathlib import Path
import os

# make sure highway.mp4 is present
video_path = Path("highway.mp4")
assert video_path.exists(), "Place 'highway.mp4' in this folder"

# open with ffmpeg plugin
reader = imageio.get_reader(str(video_path), 'ffmpeg')
meta    = reader.get_meta_data()
fps     = meta.get('fps', 30)     # fallback to 30 if missing
print(f"Video FPS: {fps}")

# parameters
target_duration = 10    # seconds
target_fps      = 10    # desired sampling rate
step = max(1, int(fps / target_fps))

# prepare output
out_dir = Path("frames0_10_10fps")
out_dir.mkdir(exist_ok=True)

# iterate frames, break after 10 s
for idx, frame in enumerate(reader):
    t = idx / fps
    if t > target_duration:
        break
    if idx % step == 0:
        imageio.imwrite(str(out_dir / f"{idx:04d}.jpg"), frame)

reader.close()
print(f"Extracted frames → {out_dir} (at ~{target_fps} FPS)")

Video FPS: 25.0
Extracted frames → frames0_10_10fps (at ~10 FPS)


In [ ]:
# %%
# 3. Load your fine-tuned YOLOv12 and run inference on those frames
from ultralytics import YOLO

best_weights = "runs/detect/curl-streetview-yolov12n/weights/best.pt"
model        = YOLO(best_weights)

results = model.predict(
    source=str(out_dir),
    imgsz=640,
    conf=0.1,
    iou=0.45,
    save=True,
    save_dir="runs/detect/first10_10fps"
)

print("✅ Inference done. Check annotated frames in runs/detect/first10_10fps")



image 1/126 /home/user/Cortana/machine-learning/frames0_10_10fps/0000.jpg: 384x640 7 cars, 10.6ms
image 2/126 /home/user/Cortana/machine-learning/frames0_10_10fps/0002.jpg: 384x640 7 cars, 11.0ms
image 3/126 /home/user/Cortana/machine-learning/frames0_10_10fps/0004.jpg: 384x640 6 cars, 12.6ms
image 4/126 /home/user/Cortana/machine-learning/frames0_10_10fps/0006.jpg: 384x640 7 cars, 11.9ms
image 5/126 /home/user/Cortana/machine-learning/frames0_10_10fps/0008.jpg: 384x640 6 cars, 11.0ms
image 6/126 /home/user/Cortana/machine-learning/frames0_10_10fps/0010.jpg: 384x640 6 cars, 12.3ms
image 7/126 /home/user/Cortana/machine-learning/frames0_10_10fps/0012.jpg: 384x640 6 cars, 12.0ms
image 8/126 /home/user/Cortana/machine-learning/frames0_10_10fps/0014.jpg: 384x640 6 cars, 12.2ms
image 9/126 /home/user/Cortana/machine-learning/frames0_10_10fps/0016.jpg: 384x640 5 cars, 12.9ms
image 10/126 /home/user/Cortana/machine-learning/frames0_10_10fps/0018.jpg: 384x640 7 cars, 13.0ms
image 11/126 /home